# Tensorflow快速入门二
## 知识点总结

- 梯度下降方法总结
- 模型保存和恢复

### 梯度下降方法总结
- Tensorflow直接计算，满足公式：
$\theta = \theta- \eta \bigtriangledown_{\theta} MSE(\theta)$
- tf.random_uniform()：在图中创建一个包含随机值的节点，类似于NumPy中的random()函数
- tf.assign()：创建一个将新值赋给变量的一个节点，为variable更新值

In [67]:
#--coding--:utf-8
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# 获取数据
housing = fetch_california_housing()
m,n = housing.data.shape
print housing.keys()
print housing.feature_names
print housing.target
print housing.DESCR


['data', 'feature_names', 'DESCR', 'target']
['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
[4.526 3.585 3.521 ... 0.923 0.847 0.894]
California housing dataset.

The original database is available from StatLib

    http://lib.stat.cmu.edu/

The data contains 20,640 observations on 9 variables.

This dataset contains the average house value as target variable
and the following input variables (features): average income,
housing average age, average rooms, average bedrooms, population,
average occupation, latitude, and longitude in that order.

References
----------

Pace, R. Kelley and Ronald Barry, Sparse Spatial Autoregressions,
Statistics and Probability Letters, 33 (1997) 291-297.




In [68]:
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]
scaled_data = scaler.fit_transform(housing.data)
data = np.c_[np.ones((m,1)),scaled_data]

# 设置参数
n_epoch = 1000
learning_rate = 0.01

# 设置placeholder
X = tf.constant(data,dtype = tf.float32,name = "X")
y = tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name='y')

theta = tf.Variable(tf.random_uniform([n+1, 1], -1, 1),name='theta')
y_pred = tf.matmul(X,theta,name='prediction')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error),name='mse') #Mean Squared Error 

# 计算梯度公式
gradient = 2/m * tf.matmul(tf.transpose(X),error)
training_op = tf.assign(theta,theta - learning_rate * gradient)

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epoch):
        if epoch % 100 == 0:
            print "Epoch",epoch, "MSE =", mse.eval()

    print 'best theta:',theta.eval()

Epoch 0 MSE = 8.405558
Epoch 100 MSE = 8.405558
Epoch 200 MSE = 8.405558
Epoch 300 MSE = 8.405558
Epoch 400 MSE = 8.405558
Epoch 500 MSE = 8.405558
Epoch 600 MSE = 8.405558
Epoch 700 MSE = 8.405558
Epoch 800 MSE = 8.405558
Epoch 900 MSE = 8.405558
best theta: [[ 0.04001427]
 [-0.37021732]
 [-0.57172775]
 [-0.23449564]
 [-0.19885969]
 [ 0.92964244]
 [ 0.18611884]
 [-0.6614673 ]
 [-0.07952976]]


### 自动求导autodiff
手动求导方法有两个缺陷：
- 深度神经网络中公式较长
- 计算效率较低
可以采取以下命令进行自动求导：
```bash
gradients = tf.gradients(mse,[theta])[0]
```
gradient有两个参数：
- op，损失函数，这里是mse
- variable lists，变量列表，这里是theta值

### 使用Optimizer
可以使用各种优化器,如梯度下降优化器
```bash
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)
```
还有很多其他优化器，例如收敛更快的MomentumOptimizer优化器等

### 梯度下降中传输数据的方式
- mini-batchbatch
- 方法：使用占位符placeholder

In [69]:
# 传统方式
A = tf.placeholder(tf.float32,shape=(None,3))
B = A + 5
with tf.Session() as sess:
    test_b_1 = B.eval(feed_dict={A:[[1,2,3]]})
    test_b_2 = B.eval(feed_dict={A:[[4,5,6],[7,8,9]]})
print(test_b_1)  
print(test_b_2) 

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


In [70]:
# 使用mini-batch方式
X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

# 与上文例子相同，这里采用的是使用optimizer计算梯度
n_epochs = 100
display_step = 10
learning_rate = 0.01
batch_size = 100
n_batches = int(np.ceil(m / batch_size))
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta") #X0
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()


# 定义mini-batch取数据方式
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index) 
    indices = np.random.randint(m, size=batch_size)  
    X_batch = data[indices] 
    y_batch = housing.target.reshape(-1, 1)[indices] 
    return X_batch, y_batch

# mini-batch计算过程
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):#迭代的次数
        avg_cost = 0.
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

            if epoch % 10 == 0:
                total_loss = 0
                acc_train = mse.eval(feed_dict={X: X_batch, y: y_batch})
                total_loss += acc_train 
                #print acc_train,total_loss
                print  "epoch:",epoch, "batch_id:",batch_index, "Batch loss:", acc_train


epoch: 0 batch_id: 0 Batch loss: 2.1672046
epoch: 0 batch_id: 1 Batch loss: 2.295659
epoch: 0 batch_id: 2 Batch loss: 1.874109
epoch: 0 batch_id: 3 Batch loss: 2.2121804
epoch: 0 batch_id: 4 Batch loss: 2.167641
epoch: 0 batch_id: 5 Batch loss: 1.7245495
epoch: 0 batch_id: 6 Batch loss: 2.0318623
epoch: 0 batch_id: 7 Batch loss: 1.5136069
epoch: 0 batch_id: 8 Batch loss: 1.6046904
epoch: 0 batch_id: 9 Batch loss: 1.3764426
epoch: 0 batch_id: 10 Batch loss: 1.3385471
epoch: 0 batch_id: 11 Batch loss: 1.7322328
epoch: 0 batch_id: 12 Batch loss: 1.4998823
epoch: 0 batch_id: 13 Batch loss: 1.9056491
epoch: 0 batch_id: 14 Batch loss: 3.0762713
epoch: 0 batch_id: 15 Batch loss: 1.2916967
epoch: 0 batch_id: 16 Batch loss: 1.5571638
epoch: 0 batch_id: 17 Batch loss: 1.598186
epoch: 0 batch_id: 18 Batch loss: 1.3730277
epoch: 0 batch_id: 19 Batch loss: 1.1644965
epoch: 0 batch_id: 20 Batch loss: 1.1785976
epoch: 0 batch_id: 21 Batch loss: 1.576241
epoch: 0 batch_id: 22 Batch loss: 1.4259119
epo

epoch: 10 batch_id: 0 Batch loss: 0.6810457
epoch: 10 batch_id: 1 Batch loss: 0.7431099
epoch: 10 batch_id: 2 Batch loss: 0.5606531
epoch: 10 batch_id: 3 Batch loss: 0.6081461
epoch: 10 batch_id: 4 Batch loss: 0.90225893
epoch: 10 batch_id: 5 Batch loss: 0.5305664
epoch: 10 batch_id: 6 Batch loss: 0.53555506
epoch: 10 batch_id: 7 Batch loss: 0.56174487
epoch: 10 batch_id: 8 Batch loss: 0.4855208
epoch: 10 batch_id: 9 Batch loss: 0.379264
epoch: 10 batch_id: 10 Batch loss: 0.5051014
epoch: 10 batch_id: 11 Batch loss: 0.5390079
epoch: 10 batch_id: 12 Batch loss: 0.7699985
epoch: 10 batch_id: 13 Batch loss: 0.45973587
epoch: 10 batch_id: 14 Batch loss: 0.5360129
epoch: 10 batch_id: 15 Batch loss: 0.6091374
epoch: 10 batch_id: 16 Batch loss: 0.4848224
epoch: 10 batch_id: 17 Batch loss: 0.64357764
epoch: 10 batch_id: 18 Batch loss: 0.39952487
epoch: 10 batch_id: 19 Batch loss: 0.5636877
epoch: 10 batch_id: 20 Batch loss: 0.5957983
epoch: 10 batch_id: 21 Batch loss: 0.55822176
epoch: 10 batc

epoch: 20 batch_id: 0 Batch loss: 0.564179
epoch: 20 batch_id: 1 Batch loss: 0.5064613
epoch: 20 batch_id: 2 Batch loss: 0.35872972
epoch: 20 batch_id: 3 Batch loss: 0.59568715
epoch: 20 batch_id: 4 Batch loss: 0.3325283
epoch: 20 batch_id: 5 Batch loss: 0.5385927
epoch: 20 batch_id: 6 Batch loss: 0.50510526
epoch: 20 batch_id: 7 Batch loss: 0.39496544
epoch: 20 batch_id: 8 Batch loss: 0.50682276
epoch: 20 batch_id: 9 Batch loss: 0.5851183
epoch: 20 batch_id: 10 Batch loss: 0.6037811
epoch: 20 batch_id: 11 Batch loss: 0.46415946
epoch: 20 batch_id: 12 Batch loss: 0.50726724
epoch: 20 batch_id: 13 Batch loss: 0.40837625
epoch: 20 batch_id: 14 Batch loss: 0.40306637
epoch: 20 batch_id: 15 Batch loss: 0.57209855
epoch: 20 batch_id: 16 Batch loss: 0.51130736
epoch: 20 batch_id: 17 Batch loss: 0.7655057
epoch: 20 batch_id: 18 Batch loss: 0.437026
epoch: 20 batch_id: 19 Batch loss: 0.46590033
epoch: 20 batch_id: 20 Batch loss: 0.58546907
epoch: 20 batch_id: 21 Batch loss: 0.5032965
epoch: 20

epoch: 30 batch_id: 0 Batch loss: 0.522121
epoch: 30 batch_id: 1 Batch loss: 0.62666833
epoch: 30 batch_id: 2 Batch loss: 0.5682728
epoch: 30 batch_id: 3 Batch loss: 0.577856
epoch: 30 batch_id: 4 Batch loss: 0.47186974
epoch: 30 batch_id: 5 Batch loss: 0.4389424
epoch: 30 batch_id: 6 Batch loss: 0.40743542
epoch: 30 batch_id: 7 Batch loss: 0.54487944
epoch: 30 batch_id: 8 Batch loss: 0.43429604
epoch: 30 batch_id: 9 Batch loss: 0.57226974
epoch: 30 batch_id: 10 Batch loss: 0.44691157
epoch: 30 batch_id: 11 Batch loss: 0.51691073
epoch: 30 batch_id: 12 Batch loss: 0.43174136
epoch: 30 batch_id: 13 Batch loss: 0.57407635
epoch: 30 batch_id: 14 Batch loss: 0.53312165
epoch: 30 batch_id: 15 Batch loss: 0.64613557
epoch: 30 batch_id: 16 Batch loss: 0.701489
epoch: 30 batch_id: 17 Batch loss: 0.55357325
epoch: 30 batch_id: 18 Batch loss: 0.6624898
epoch: 30 batch_id: 19 Batch loss: 0.42069098
epoch: 30 batch_id: 20 Batch loss: 0.33179688
epoch: 30 batch_id: 21 Batch loss: 0.5125233
epoch: 3

epoch: 40 batch_id: 0 Batch loss: 0.63110894
epoch: 40 batch_id: 1 Batch loss: 0.44274566
epoch: 40 batch_id: 2 Batch loss: 0.4170633
epoch: 40 batch_id: 3 Batch loss: 0.46116486
epoch: 40 batch_id: 4 Batch loss: 0.5063678
epoch: 40 batch_id: 5 Batch loss: 0.4987097
epoch: 40 batch_id: 6 Batch loss: 0.60123754
epoch: 40 batch_id: 7 Batch loss: 0.68144166
epoch: 40 batch_id: 8 Batch loss: 0.6921921
epoch: 40 batch_id: 9 Batch loss: 0.46396896
epoch: 40 batch_id: 10 Batch loss: 0.5143482
epoch: 40 batch_id: 11 Batch loss: 0.6211417
epoch: 40 batch_id: 12 Batch loss: 0.43429863
epoch: 40 batch_id: 13 Batch loss: 0.4815974
epoch: 40 batch_id: 14 Batch loss: 0.69119763
epoch: 40 batch_id: 15 Batch loss: 0.6209749
epoch: 40 batch_id: 16 Batch loss: 0.43140128
epoch: 40 batch_id: 17 Batch loss: 0.6167991
epoch: 40 batch_id: 18 Batch loss: 0.39349478
epoch: 40 batch_id: 19 Batch loss: 0.3253084
epoch: 40 batch_id: 20 Batch loss: 0.49223077
epoch: 40 batch_id: 21 Batch loss: 0.48033088
epoch: 4

epoch: 50 batch_id: 0 Batch loss: 0.6762616
epoch: 50 batch_id: 1 Batch loss: 0.46353003
epoch: 50 batch_id: 2 Batch loss: 0.41833317
epoch: 50 batch_id: 3 Batch loss: 0.7471763
epoch: 50 batch_id: 4 Batch loss: 0.45071954
epoch: 50 batch_id: 5 Batch loss: 0.54192644
epoch: 50 batch_id: 6 Batch loss: 0.5483677
epoch: 50 batch_id: 7 Batch loss: 0.5675137
epoch: 50 batch_id: 8 Batch loss: 0.48332444
epoch: 50 batch_id: 9 Batch loss: 0.4323386
epoch: 50 batch_id: 10 Batch loss: 0.43754783
epoch: 50 batch_id: 11 Batch loss: 0.47886688
epoch: 50 batch_id: 12 Batch loss: 0.6657603
epoch: 50 batch_id: 13 Batch loss: 0.49313942
epoch: 50 batch_id: 14 Batch loss: 0.42447922
epoch: 50 batch_id: 15 Batch loss: 0.3617839
epoch: 50 batch_id: 16 Batch loss: 0.43233162
epoch: 50 batch_id: 17 Batch loss: 0.52427655
epoch: 50 batch_id: 18 Batch loss: 0.38027176
epoch: 50 batch_id: 19 Batch loss: 0.44835252
epoch: 50 batch_id: 20 Batch loss: 0.37160653
epoch: 50 batch_id: 21 Batch loss: 0.5155723
epoch:

epoch: 60 batch_id: 0 Batch loss: 0.6542897
epoch: 60 batch_id: 1 Batch loss: 0.4686804
epoch: 60 batch_id: 2 Batch loss: 0.54337174
epoch: 60 batch_id: 3 Batch loss: 0.6088903
epoch: 60 batch_id: 4 Batch loss: 0.9116748
epoch: 60 batch_id: 5 Batch loss: 0.5355449
epoch: 60 batch_id: 6 Batch loss: 0.38619727
epoch: 60 batch_id: 7 Batch loss: 0.63734347
epoch: 60 batch_id: 8 Batch loss: 0.6920177
epoch: 60 batch_id: 9 Batch loss: 0.5993974
epoch: 60 batch_id: 10 Batch loss: 0.5320807
epoch: 60 batch_id: 11 Batch loss: 0.35805434
epoch: 60 batch_id: 12 Batch loss: 0.4077484
epoch: 60 batch_id: 13 Batch loss: 0.700916
epoch: 60 batch_id: 14 Batch loss: 0.56255525
epoch: 60 batch_id: 15 Batch loss: 0.77477616
epoch: 60 batch_id: 16 Batch loss: 0.4382529
epoch: 60 batch_id: 17 Batch loss: 0.3023141
epoch: 60 batch_id: 18 Batch loss: 0.6116199
epoch: 60 batch_id: 19 Batch loss: 0.56594205
epoch: 60 batch_id: 20 Batch loss: 0.9390855
epoch: 60 batch_id: 21 Batch loss: 0.39617738
epoch: 60 bat

epoch: 70 batch_id: 0 Batch loss: 0.45179355
epoch: 70 batch_id: 1 Batch loss: 0.4507701
epoch: 70 batch_id: 2 Batch loss: 0.4743873
epoch: 70 batch_id: 3 Batch loss: 0.41053924
epoch: 70 batch_id: 4 Batch loss: 0.54216146
epoch: 70 batch_id: 5 Batch loss: 0.454568
epoch: 70 batch_id: 6 Batch loss: 0.44388306
epoch: 70 batch_id: 7 Batch loss: 0.59457433
epoch: 70 batch_id: 8 Batch loss: 0.39245114
epoch: 70 batch_id: 9 Batch loss: 0.46223447
epoch: 70 batch_id: 10 Batch loss: 0.39544106
epoch: 70 batch_id: 11 Batch loss: 0.34748986
epoch: 70 batch_id: 12 Batch loss: 0.616122
epoch: 70 batch_id: 13 Batch loss: 0.51772577
epoch: 70 batch_id: 14 Batch loss: 0.78781325
epoch: 70 batch_id: 15 Batch loss: 0.49549904
epoch: 70 batch_id: 16 Batch loss: 0.53694403
epoch: 70 batch_id: 17 Batch loss: 0.64441025
epoch: 70 batch_id: 18 Batch loss: 0.32865158
epoch: 70 batch_id: 19 Batch loss: 0.7547708
epoch: 70 batch_id: 20 Batch loss: 0.47380447
epoch: 70 batch_id: 21 Batch loss: 0.5900756
epoch:

epoch: 80 batch_id: 0 Batch loss: 0.49124497
epoch: 80 batch_id: 1 Batch loss: 0.5629084
epoch: 80 batch_id: 2 Batch loss: 0.65633446
epoch: 80 batch_id: 3 Batch loss: 0.4944587
epoch: 80 batch_id: 4 Batch loss: 0.51620936
epoch: 80 batch_id: 5 Batch loss: 0.5261065
epoch: 80 batch_id: 6 Batch loss: 0.38446182
epoch: 80 batch_id: 7 Batch loss: 0.4790051
epoch: 80 batch_id: 8 Batch loss: 0.52570593
epoch: 80 batch_id: 9 Batch loss: 0.6171104
epoch: 80 batch_id: 10 Batch loss: 0.3033197
epoch: 80 batch_id: 11 Batch loss: 0.34371358
epoch: 80 batch_id: 12 Batch loss: 0.36458832
epoch: 80 batch_id: 13 Batch loss: 0.32325545
epoch: 80 batch_id: 14 Batch loss: 0.5265043
epoch: 80 batch_id: 15 Batch loss: 0.51269126
epoch: 80 batch_id: 16 Batch loss: 0.44695374
epoch: 80 batch_id: 17 Batch loss: 0.37609553
epoch: 80 batch_id: 18 Batch loss: 0.5804351
epoch: 80 batch_id: 19 Batch loss: 0.50121933
epoch: 80 batch_id: 20 Batch loss: 0.49805725
epoch: 80 batch_id: 21 Batch loss: 0.4156387
epoch: 

epoch: 90 batch_id: 0 Batch loss: 0.5466644
epoch: 90 batch_id: 1 Batch loss: 0.58521575
epoch: 90 batch_id: 2 Batch loss: 0.64257616
epoch: 90 batch_id: 3 Batch loss: 0.2374793
epoch: 90 batch_id: 4 Batch loss: 0.5331865
epoch: 90 batch_id: 5 Batch loss: 0.50250727
epoch: 90 batch_id: 6 Batch loss: 0.5760769
epoch: 90 batch_id: 7 Batch loss: 0.44275123
epoch: 90 batch_id: 8 Batch loss: 0.5203735
epoch: 90 batch_id: 9 Batch loss: 0.38756168
epoch: 90 batch_id: 10 Batch loss: 0.5600635
epoch: 90 batch_id: 11 Batch loss: 0.76658714
epoch: 90 batch_id: 12 Batch loss: 0.42658195
epoch: 90 batch_id: 13 Batch loss: 0.64094
epoch: 90 batch_id: 14 Batch loss: 0.6335204
epoch: 90 batch_id: 15 Batch loss: 0.666764
epoch: 90 batch_id: 16 Batch loss: 0.74403644
epoch: 90 batch_id: 17 Batch loss: 0.6247132
epoch: 90 batch_id: 18 Batch loss: 0.7048914
epoch: 90 batch_id: 19 Batch loss: 0.72463655
epoch: 90 batch_id: 20 Batch loss: 0.5092488
epoch: 90 batch_id: 21 Batch loss: 0.7428656
epoch: 90 batc

## 模型的保存和恢复
- 在创建图阶段创建一个Saver的节点，在执行阶段需要保存模型的地方调用Save()函数即可

In [ ]:
# 模型的保存
init = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())  # 保存运行过程
            save_path = saver.save(sess, "/my_model.ckpt")
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "/my_model_final.ckpt")#保存最后的结果
    
# 模型的加载
with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    best_theta_restored = theta.eval() 
    
saver = tf.train.Saver({"weights":theta})


reset_graph()
saver = tf.train.import_meta_graph("/tmp/my_model_final.ckpt.meta")
theta = tf.get_default_graph().get_tensor_by_name("theta:0")
